In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn_pandas import DataFrameMapper
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.tree import export_graphviz
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

## 模型的apply函数

In [2]:
application_train = pd.read_csv('data/application_train.csv')

X = application_train[[col for col in application_train.columns if col not in ['SK_ID_CURR', 'TARGET', 'DATE']]]
y = application_train['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

col_types = X_train.dtypes
char_cols = list(col_types[col_types == 'object'].index)
num_cols = list(col_types[col_types != 'object'].index)

# GradientBoostingClassifier不支持缺失，填充缺失
X_train[char_cols] = X_train[char_cols].fillna('null')
X_train[num_cols] = X_train[num_cols].fillna(-1)
X_test[char_cols] = X_test[char_cols].fillna('null')
X_test[num_cols] = X_test[num_cols].fillna(-1)

# GradientBoostingClassifier不支持类别特征，对类别特征进行ont-hot编码，并使测试集与训练集编码后的特征保持一致
X_train_new = X_train[num_cols].join(pd.get_dummies(X_train[char_cols]))
X_test_new = X_test[num_cols].join(pd.get_dummies(X_test[char_cols]))

for col in set(X_train_new.columns) - set(X_test_new.columns):
    X_test_new[col] = 0
X_test_new = X_test_new[X_train_new.columns]

# 训练模型
model = GradientBoostingClassifier(random_state=1234)
model.fit(X_train_new, y_train)

# 获取落到每颗子树的叶子节点结果
train_leaf_feature1 = model.apply(X_train_new)[:, :, 0]
test_leaf_feature1 = model.apply(X_test_new)[:, :, 0]

# 转换成GBDT特征
enc = OneHotEncoder()
enc.fit(train_leaf_feature1)
train_gbdt_feature1 = np.array(enc.transform(train_leaf_feature1).toarray())
test_gbdt_feature1 = np.array(enc.transform(test_leaf_feature1).toarray())

In [3]:
train_leaf_feature1

array([[13., 13., 13., ...,  3.,  4., 14.],
       [13., 14., 14., ...,  6.,  3., 14.],
       [10., 10., 11., ...,  3.,  4., 14.],
       ...,
       [13., 13., 13., ..., 11.,  4., 14.],
       [ 6.,  7.,  7., ...,  3.,  4., 14.],
       [ 4.,  4., 10., ..., 10.,  4., 14.]])

In [4]:
train_leaf_feature1.shape

(43051, 100)

In [5]:
import graphviz
dot_data = export_graphviz(model.estimators_[0, 0], out_file=None, feature_names=X_train_new.columns,
                           filled=True, rounded=True, special_characters=True)
graph = graphviz.Source(dot_data)
graph.render('tree1.gv')

'tree1.gv.pdf'

In [6]:
set(train_leaf_feature1[:, 0])

{3.0, 4.0, 6.0, 7.0, 10.0, 11.0, 13.0, 14.0}

In [7]:
X_train.iloc[0][['EXT_SOURCE_2', 'EXT_SOURCE_3']]

EXT_SOURCE_2    0.447966
EXT_SOURCE_3     0.58674
Name: 53107, dtype: object

In [8]:
train_gbdt_feature1[0, :8]

array([0., 0., 0., 0., 0., 0., 1., 0.])

## 解析树结构

In [9]:
export_graphviz(model.estimators_[0, 0], feature_names=X_train_new.columns, precision=6).split('\n')

['digraph Tree {',
 'node [shape=box] ;',
 '0 [label="EXT_SOURCE_2 <= 0.356838\\nfriedman_mse = 0.074066\\nsamples = 43051\\nvalue = 0.0"] ;',
 '1 [label="EXT_SOURCE_3 <= 0.36813\\nfriedman_mse = 0.127764\\nsamples = 9423\\nvalue = 0.069821"] ;',
 '0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;',
 '2 [label="EXT_SOURCE_2 <= 0.108422\\nfriedman_mse = 0.165696\\nsamples = 4436\\nvalue = 0.129093"] ;',
 '1 -> 2 ;',
 '3 [label="friedman_mse = 0.213775\\nsamples = 972\\nvalue = 3.093375"] ;',
 '2 -> 3 ;',
 '4 [label="friedman_mse = 0.14861\\nsamples = 3464\\nvalue = 1.363997"] ;',
 '2 -> 4 ;',
 '5 [label="EXT_SOURCE_2 <= 0.137969\\nfriedman_mse = 0.088118\\nsamples = 4987\\nvalue = 0.017098"] ;',
 '1 -> 5 ;',
 '6 [label="friedman_mse = 0.124332\\nsamples = 1134\\nvalue = 0.876876"] ;',
 '5 -> 6 ;',
 '7 [label="friedman_mse = 0.076587\\nsamples = 3853\\nvalue = 0.040715"] ;',
 '5 -> 7 ;',
 '8 [label="EXT_SOURCE_3 <= 0.42325\\nfriedman_mse = 0.057271\\nsamples = 33628\\nvalue =

In [10]:
now_col = 'NAME_TYPE_SUITE_Other_A'
char_col = 'NAME_TYPE_SUITE'

In [11]:
def get_gbdt_path(model, data_cols, char_cols=[], precision=6):
    '''
    根据数据集和GBDT模型,输出按照GBDT的路径衍生的变量:
    ------------------------------------------------------------
    入参结果如下:
        model: 训练完的GBDT模型
        data_cols: 训练数据集的特征列表
        char_cols: 类别特征的特征列表
            由于类别特征需one-hot编码后进模型,在树结构中会出现var_value > 0.5的节点,传入char_cols则可以将节点改为var ==/!= value
        precision: 提取每个子树的文字结构时,内部节点判断阈值的精确小数位，保留位数太少可能存在误差
    ------------------------------------------------------------
    出参结果如下:
        gbdt_path: GBDT所有叶子节点经过的路径
    '''
    # 类别特征节点判断转换的字段
    if isinstance(char_cols, list) and char_cols != []:
        node_judge_trans = {}
        for char_col in char_cols:
            for now_col in data_cols:
                if char_col in now_col:
                    node_judge_trans['{} > 0.5'.format(now_col)] = '{} == {}'.format(char_col, now_col.replace(char_col + '_', ''))
                    node_judge_trans['{} <= 0.5'.format(now_col)] = '{} != {}'.format(char_col, now_col.replace(char_col + '_', ''))
    
    # 所有树的叶子节点路径
    leaf_node_path = []
    # 遍历每棵子树
    for n in range(model.n_estimators_):
        tree_text = export_graphviz(model.estimators_[n, 0], feature_names=data_cols, precision=precision)

        # 节点
        node = {}
        # 叶子节点
        leaf_node_index = []
        # 节点经过的路径
        node_path = {}
        # 分裂的节点
        split_node = []

        # 遍历每颗子树内部情况
        for line in tree_text.split('\n'):
            # 获取现节点的索引
            if '[label=' in line:
                node_index = int(line[: line.find('[') - 1])
                # 如果现节点有条件判断，保存判断
                if 'label="friedman_mse' not in line:
                    node[node_index] = line[line.find('label="') + 7: line.find('\\nfriedman_mse')]
                else:
                    leaf_node_index.append(node_index)

            # 上一个节点到下一个节点的路径
            if '->' in line:
                # 获取上一个节点的索引
                previous_node_index = int(line[: line.find('->') - 1])
                # 获取下一个节点的索引
                if '[' in line:
                    next_node_index = int(line[line.find('->') + 3: line.find('[') - 1])
                else:
                    next_node_index = int(line[line.find('->') + 3: line.find(';') - 1])

                # 分裂的左路径
                if previous_node_index not in split_node:
                    split_node.append(previous_node_index)
                    # 从根节点到现节点的路径
                    if previous_node_index == 0:
                        node_path[next_node_index] = node[previous_node_index]
                    else:
                        node_path[next_node_index] = (node_path[previous_node_index] + ' & ' + node[previous_node_index])
                # 分裂的右路径
                else:
                    # 从根节点到现节点的路径
                    if previous_node_index == 0:
                        node_path[next_node_index] = node[previous_node_index].replace('<=', '>')
                    else:
                        node_path[next_node_index] = (node_path[previous_node_index] + ' & ' + node[previous_node_index].replace('<=', '>'))
                        
        if isinstance(char_cols, list) and char_cols != []:
            # 遍历刚才生成的每个节点对应路径的字典
            for index in node_path.keys():
                # 由于类别特征会进行one-hot编码,导致节点的判断为<=0.5/>0.5,将其替换成未one-hot编码前的表达方式var ==/!= value
                for old_judge, new_judge in node_judge_trans.items():
                    node_path[index] = node_path[index].replace(old_judge, new_judge)

        leaf_node_path += [value for key, value in node_path.items() if key in leaf_node_index]
    
    gbdt_path = {index + 1: value for index, value in enumerate(leaf_node_path)}
    
    return gbdt_path

In [12]:
data_cols = X_train_new.columns
gbdt_path = get_gbdt_path(model, data_cols, char_cols, precision=6)

In [13]:
print('模型的apply函数方法提取的GBDT特征数：', train_gbdt_feature1.shape[1])
print('解析树结构方法提取的GBDT路径数：', len(gbdt_path))

模型的apply函数方法提取的GBDT特征数： 773
解析树结构方法提取的GBDT路径数： 773


In [33]:
{key:value for key, value in gbdt_path.items() if key in range(1, 11)}

{1: 'EXT_SOURCE_2 <= 0.356838 & EXT_SOURCE_3 <= 0.36813 & EXT_SOURCE_2 <= 0.108422',
 2: 'EXT_SOURCE_2 <= 0.356838 & EXT_SOURCE_3 <= 0.36813 & EXT_SOURCE_2 > 0.108422',
 3: 'EXT_SOURCE_2 <= 0.356838 & EXT_SOURCE_3 > 0.36813 & EXT_SOURCE_2 <= 0.137969',
 4: 'EXT_SOURCE_2 <= 0.356838 & EXT_SOURCE_3 > 0.36813 & EXT_SOURCE_2 > 0.137969',
 5: 'EXT_SOURCE_2 > 0.356838 & EXT_SOURCE_3 <= 0.42325 & EXT_SOURCE_2 <= 0.597194',
 6: 'EXT_SOURCE_2 > 0.356838 & EXT_SOURCE_3 <= 0.42325 & EXT_SOURCE_2 > 0.597194',
 7: 'EXT_SOURCE_2 > 0.356838 & EXT_SOURCE_3 > 0.42325 & EXT_SOURCE_2 <= 0.623373',
 8: 'EXT_SOURCE_2 > 0.356838 & EXT_SOURCE_3 > 0.42325 & EXT_SOURCE_2 > 0.623373',
 9: 'EXT_SOURCE_2 <= 0.356838 & EXT_SOURCE_3 <= 0.373177 & EXT_SOURCE_2 <= 0.106543',
 10: 'EXT_SOURCE_2 <= 0.356838 & EXT_SOURCE_3 <= 0.373177 & EXT_SOURCE_2 > 0.106543'}

In [15]:
def get_gbdt_feature(data, gbdt_path, cols_name=1):
    '''
    根据get_gbdt_path_var返回数据集的变量列表,回溯gbdt衍生给其他数据集:
    ------------------------------------------------------------
    入参结果如下:
        data: 需衍生变量的数据集,pd.DataFrame对象,不能有缺失
        gbdt_cols: get_gbdt_path返回的GBDT所有叶子节点的对应路径
        cols_name：是否使用GBDT路径作为列名,默认为1,传入0则将列名用叶子节点序号代替
    ------------------------------------------------------------
    出参结果如下:
        gbdt_feature: GBDT特征
    '''
    gbdt_feature = pd.DataFrame()

    # 遍历所有叶子节点经过的路径提取GBDT特征
    for key, value in gbdt_path.items():
        # 需执行exec的字符串开头
        if cols_name == 1:
            exec_str = "gbdt_feature['{}'] = ".format(value)
        elif cols_name == 0:
            exec_str = "gbdt_feature['{}'] = ".format(key)
        else:
            raise ValueError('请传入正确的cols_name参数,cols_name应为1或0')
            
        judge_len = len(value.split(' & '))

        # 遍历变量名中的所有条件加到exec字符串中
        for index, judge in enumerate(value.split(' & ')):
            # 数值型的条件
            if ('!=' not in judge) and ('==' not in judge):
                # 只有一个条件
                if judge_len == 1:
                    exec_str += ("(data['{}']{})".format(judge[: judge.find(' ')], judge[judge.find(' '):]))
                # 第一个条件
                elif index == 0:
                    exec_str += ("((data['{}']{})".format(judge[: judge.find(' ')], judge[judge.find(' '):]))
                # 最后一个条件
                elif index == judge_len - 1:
                    exec_str += (" & (data['{}']{}))".format(judge[: judge.find(' ')], judge[judge.find(' '):]))
                # 中间的条件
                else:
                    exec_str += (" & (data['{}']{})".format(judge[: judge.find(' ')], judge[judge.find(' '):]))
            # 字符型的条件
            else:
                if '==' in judge:
                    offset = 3
                else:
                    offset = 2
                # 只有一个条件
                if judge_len == 1:
                    exec_str += ("(data['{}']{}'{}')".format(judge[: judge.find(' ')],
                        judge[judge.find(' '): judge.find('=') + offset], judge[judge.find('=') + offset:]))
                # 第一个条件
                elif index == 0:
                    exec_str += ("((data['{}']{}'{}')".format(judge[: judge.find(' ')],
                        judge[judge.find(' '): judge.find('=') + offset], judge[judge.find('=') + offset:]))
                # 最后一个条件
                elif index == judge_len - 1:
                    exec_str += (" & (data['{}']{}'{}'))".format(judge[: judge.find(' ')], 
                        judge[judge.find(' '): judge.find('=') + offset], judge[judge.find('=') + offset:]))
                # 中间的条件
                else:
                    exec_str += (" & (data['{}']{}'{}')".format(judge[: judge.find(' ')], 
                        judge[judge.find(' '): judge.find('=') + offset], judge[judge.find('=') + offset:]))

        # 将True/False替换成1/0
        exec_str += ".astype(int)"
        
        exec(exec_str)

    return gbdt_feature

In [16]:
# 由于类别型的判断转换成了one-hot编码前的判断，可以用未经过one-hot编码的原数据集
train_gbdt_feature2 = get_gbdt_feature(X_train, gbdt_path)
train_gbdt_feature2.head()

,EXT_SOURCE_2 <= 0.356838 & EXT_SOURCE_3 <= 0.36813 & EXT_SOURCE_2 <= 0.108422,EXT_SOURCE_2 <= 0.356838 & EXT_SOURCE_3 <= 0.36813 & EXT_SOURCE_2 > 0.108422,EXT_SOURCE_2 <= 0.356838 & EXT_SOURCE_3 > 0.36813 & EXT_SOURCE_2 <= 0.137969,EXT_SOURCE_2 <= 0.356838 & EXT_SOURCE_3 > 0.36813 & EXT_SOURCE_2 > 0.137969,EXT_SOURCE_2 > 0.356838 & EXT_SOURCE_3 <= 0.42325 & EXT_SOURCE_2 <= 0.597194,EXT_SOURCE_2 > 0.356838 & EXT_SOURCE_3 <= 0.42325 & EXT_SOURCE_2 > 0.597194,EXT_SOURCE_2 > 0.356838 & EXT_SOURCE_3 > 0.42325 & EXT_SOURCE_2 <= 0.623373,EXT_SOURCE_2 > 0.356838 & EXT_SOURCE_3 > 0.42325 & EXT_SOURCE_2 > 0.623373,EXT_SOURCE_2 <= 0.356838 & EXT_SOURCE_3 <= 0.373177 & EXT_SOURCE_2 <= 0.106543,EXT_SOURCE_2 <= 0.356838 & EXT_SOURCE_3 <= 0.373177 & EXT_SOURCE_2 > 0.106543,...,LIVINGAPARTMENTS_AVG <= 0.1507 & NAME_INCOME_TYPE == State servant & NONLIVINGAPARTMENTS_MODE <= 0.18285,LIVINGAPARTMENTS_AVG <= 0.1507 & NAME_INCOME_TYPE == State servant & NONLIVINGAPARTMENTS_MODE > 0.18285,LIVINGAPARTMENTS_AVG > 0.1507 & DEF_60_CNT_SOCIAL_CIRCLE <= 1.5 & HOUSETYPE_MODE != terraced house,LIVINGAPARTMENTS_AVG > 0.1507 & DEF_60_CNT_SOCIAL_CIRCLE <= 1.5 & HOUSETYPE_MODE == terraced house,LIVINGAPARTMENTS_AVG > 0.1507 & DEF_60_CNT_SOCIAL_CIRCLE > 1.5 & YEARS_BUILD_MODE <= 0.8138,LIVINGAPARTMENTS_AVG > 0.1507 & DEF_60_CNT_SOCIAL_CIRCLE > 1.5 & YEARS_BUILD_MODE > 0.8138,EXT_SOURCE_3 <= 0.015787 & WEEKDAY_APPR_PROCESS_START != TUESDAY & AMT_REQ_CREDIT_BUREAU_YEAR <= 10.0,EXT_SOURCE_3 <= 0.015787 & WEEKDAY_APPR_PROCESS_START != TUESDAY & AMT_REQ_CREDIT_BUREAU_YEAR > 10.0,EXT_SOURCE_3 > 0.015787 & EXT_SOURCE_3 <= 0.058627 & OCCUPATION_TYPE != Core staff,EXT_SOURCE_3 > 0.015787 & EXT_SOURCE_3 <= 0.058627 & OCCUPATION_TYPE == Core staff
53107,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42038,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48694,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39556,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
25479,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [17]:
print('模型的apply函数提取的GBDT特征维度', train_gbdt_feature1.shape)
print('解析树结构提取的GBDT特征维度', train_gbdt_feature2.shape)

模型的apply函数提取的GBDT特征维度 (43051, 773)
解析树结构提取的GBDT特征维度 (43051, 745)


In [18]:
train_gbdt_feature3 = get_gbdt_feature(X_train, gbdt_path, cols_name=0)
test_gbdt_feature3 = get_gbdt_feature(X_test, gbdt_path, cols_name=0)

print('训练集：')
print('模型的apply函数提取的GBDT特征维度：', train_gbdt_feature1.shape)
print('解析树结构提取的GBDT特征维度：', train_gbdt_feature3.shape)
print('两个GBDT特征的所有值是否完全相等：', (train_gbdt_feature1 == np.array(train_gbdt_feature3)).all())

print('测试集：')
print('模型的apply函数提取的GBDT特征维度：', test_gbdt_feature1.shape)
print('解析树结构提取的GBDT特征维度：', test_gbdt_feature3.shape)
print('两个GBDT特征的所有值是否完全相等：', (test_gbdt_feature1 == np.array(test_gbdt_feature3)).all())

训练集：
模型的apply函数提取的GBDT特征维度： (43051, 773)
解析树结构提取的GBDT特征维度： (43051, 773)
两个GBDT特征的所有值是否完全相等： True
测试集：
模型的apply函数提取的GBDT特征维度： (18451, 773)
解析树结构提取的GBDT特征维度： (18451, 773)
两个GBDT特征的所有值是否完全相等： False


In [19]:
test_gbdt_feature3.iloc[(test_gbdt_feature1 != np.array(test_gbdt_feature3)).sum(axis=1).astype(bool),
                        (test_gbdt_feature1 != np.array(test_gbdt_feature3)).sum(axis=0).astype(bool)]

,45,46
57072,0,1


In [20]:
test_gbdt_feature1[(test_gbdt_feature1 != np.array(test_gbdt_feature3)).sum(axis=1).astype(bool), 
                   (test_gbdt_feature1 != np.array(test_gbdt_feature3)).sum(axis=0).astype(bool)]

array([1., 0.])

In [21]:
gbdt_path[45]

'EXT_SOURCE_2 > 0.247911 & EXT_SOURCE_3 <= 0.425012 & EXT_SOURCE_2 <= 0.591941'

In [22]:
gbdt_path[46]

'EXT_SOURCE_2 > 0.247911 & EXT_SOURCE_3 <= 0.425012 & EXT_SOURCE_2 > 0.591941'

In [23]:
X_test.loc[57072, 'EXT_SOURCE_2']

0.5919414304289855

In [24]:
gbdt_path4 = get_gbdt_path(model, data_cols, char_cols, precision=8)

test_gbdt_feature4 = get_gbdt_feature(X_test, gbdt_path4, cols_name=0)

print('测试集：')
print('模型的apply函数提取的GBDT特征维度：', test_gbdt_feature1.shape)
print('解析树结构提取的GBDT特征维度：', test_gbdt_feature4.shape)
print('两个GBDT特征的所有值是否完全相等：', (test_gbdt_feature1 == np.array(test_gbdt_feature4)).all())

测试集：
模型的apply函数提取的GBDT特征维度： (18451, 773)
解析树结构提取的GBDT特征维度： (18451, 773)
两个GBDT特征的所有值是否完全相等： True


## GBDT特征应用

In [27]:
# 每条规则的最多使用3个特征，每条规则最少覆盖10%的样本，并且生成的规则个数不超过400个
model_rule = GradientBoostingClassifier(max_depth=3, min_samples_leaf=0.1, n_estimators=50, random_state=1234)
model_rule.fit(X_train_new, y_train)

gbdt_rule = get_gbdt_path(model_rule, data_cols, char_cols, precision=8)

train_gbdt_rule = get_gbdt_feature(X_train, gbdt_rule)

rule_df = pd.DataFrame(columns=['rule', 'cover', 'target'], index=range(train_gbdt_rule.shape[1]))
rule_df['rule'] = train_gbdt_rule.columns
data_count = len(y_train)
rule_df['cover'] = [round((train_gbdt_rule[rule] == 1).sum() / data_count, 4)
                    for rule in train_gbdt_rule.columns]
rule_df['target'] = [round(y_train[train_gbdt_rule[rule] == 1].sum() / (train_gbdt_rule[rule] == 1).sum(), 4)
                     for rule in train_gbdt_rule.columns]

rule_df = rule_df.sort_values('target', ascending=False)
rule_df.index = range(rule_df.shape[0])

print('总样本目标为 1 的样本占比为: {:.2f}%\n'.format(100 * y_train.sum() / data_count))
print('根据目标占比,提取目标占比前 5 的规则,具体规则如下:')
for index in range(5):
    print('>> 规则 {}： 覆盖率 {:.2f}%  目标占比 {:.2f}%'.format(index + 1, 100 * rule_df.loc[index, 'cover'], 100 * rule_df.loc[index, 'target']))
    print('   {}\n'.format(rule_df.loc[index, 'rule']))

总样本目标为 1 的样本占比为: 8.06%

根据目标占比,提取目标占比前 5 的规则,具体规则如下:
>> 规则 1： 覆盖率 10.30%  目标占比 20.96%
   EXT_SOURCE_2 <= 0.3568377 & EXT_SOURCE_3 <= 0.36812986

>> 规则 2： 覆盖率 11.61%  目标占比 20.22%
   EXT_SOURCE_3 <= 0.39019689 & EXT_SOURCE_2 <= 0.37501432

>> 规则 3： 覆盖率 10.05%  目标占比 19.89%
   EXT_SOURCE_1 <= 0.42324439 & DAYS_BIRTH > -19935.5 & EXT_SOURCE_2 <= 0.28651884

>> 规则 4： 覆盖率 12.61%  目标占比 19.43%
   EXT_SOURCE_2 <= 0.48394567 & EXT_SOURCE_3 <= 0.26575312

>> 规则 5： 覆盖率 10.85%  目标占比 19.36%
   EXT_SOURCE_3 <= 0.26434812 & EXT_SOURCE_3 > -0.49973637

